In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'missingno'

In [ ]:
dg_df = pd.read_csv('../big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/Demographics.csv')
dg_df.head(16)

,ID,Gender,HbA1c
0,13,MALE,5.7
1,1,FEMALE,5.5
2,3,FEMALE,5.9
3,4,FEMALE,6.4
4,5,FEMALE,5.7
5,2,MALE,5.6
6,6,FEMALE,5.8
7,7,FEMALE,5.3
8,8,FEMALE,5.6
9,10,FEMALE,6.0


In [ ]:
def calculate_acc_mag(acc_df):
    acc_x = acc_df[' acc_x']
    acc_y = acc_df[' acc_y']
    acc_z = acc_df[' acc_z']
    acc_mag = np.sqrt(acc_x ** 2 + acc_y ** 2 + acc_z ** 2)
    return acc_mag

: 

In [ ]:
# 연속적인 결측치 구간 식별
missing_ranges = []

for id in range(1, len(dg_df)+1):
    if id == 15:
        continue
    df = pd.read_csv(f'../big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/{id:03}/ACC_{id:03}.csv')

    print(f'[ID: {id}] Now processing ...')

    # 데이터 처리
    df['datetime'] = pd.to_datetime(df['datetime'])

    df['vector_mag'] = calculate_acc_mag(df)
    df.drop(columns=[' acc_x', ' acc_y', ' acc_z'], inplace=True)

    # 전체 시간 범위를 250ms 간격으로 생성
    full_time_range = pd.date_range(start=df['datetime'].min(), end=df['datetime'].max(), freq='31.25ms')

    # 누락된 시간대 찾기
    missing_times = full_time_range.difference(df['datetime'])

    if not missing_times.empty:
        start_time = missing_times[0]
        for i in range(1, len(missing_times)):
            if missing_times[i] != missing_times[i-1] + pd.Timedelta('31.25ms'):
                end_time = missing_times[i-1]
                #missing_ranges.append((start_time, end_time, (end_time - start_time) / pd.Timedelta('250ms') + 1))
                missing_ranges.append({
                    'id': id,
                    'start': start_time,
                    'end': end_time,
                    'count': int((end_time - start_time) / pd.Timedelta('31.25ms') + 1)
                })
                start_time = missing_times[i]
        # 마지막 구간 추가
        end_time = missing_times[-1]
        #missing_ranges.append((start_time, end_time, (end_time - start_time) / pd.Timedelta('250ms') + 1))
        missing_ranges.append({
            'id': id,
            'start': start_time,
            'end': end_time,
            'count': int((end_time - start_time) / pd.Timedelta('31.25ms') + 1)
        })

# 결과를 데이터프레임으로 변환
missing_ranges_df = pd.DataFrame(missing_ranges)

# CSV 파일로 저장
missing_ranges_df.to_csv('acc_missing_ranges.csv', index=False)

[ID: 1] Now processing ...
[ID: 2] Now processing ...
[ID: 3] Now processing ...
[ID: 4] Now processing ...
[ID: 5] Now processing ...
[ID: 6] Now processing ...
